In [44]:
import h5py
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

# Preprocess battery data from mat

# Load data
work_folder_path = 'D:\Battery_Problem'
battery_prj_path = os.path.join(
    work_folder_path, 'data-driven-prediction-of-battery-cycle-life-before-capacity-degradation-master')
f = h5py.File(os.path.join(work_folder_path, battery_prj_path, 'Data',
                           'battery_batchdata_combined.mat'))

# Extract data
output_cycles = []
output_battery_infos = []

In [45]:
def get_batch_cell_as_string(batch_cell):
    return ''.join(np.char.mod('%c', np.hstack(batch_cell)))


def pop_battery_info(battery_order, policy, policy_readable, barcode, cycle_life, cycle_count):
    output_battery_infos.append({
        'battery_order': battery_order,
        'policy': policy,
        'policy_readable': policy_readable,
        'barcode': barcode,
        'cycle_life': cycle_life,
        'cycle_count': cycle_count,
    })


def print_battery_info_csv(path):
    with open(path, 'w') as f:
        f.write(
            'battery_order,policy,policy_readable,barcode,cycle_life,cycle_count\n')
        for battery_info in output_battery_infos:
            f.write(f"{battery_info['battery_order']},{battery_info['policy']},{battery_info['policy_readable']},{battery_info['barcode']},{battery_info['cycle_life']},{battery_info['cycle_count']}\n")


def pop_cycle(battery_order, cycle_order, c1a_I_dt, c1a_avg_T, c1a_avg_I,
              c1_max_I, c2_max_I, c1_max_T, c1_min_T, c2_max_T, c2_min_T, Qi):
    output_cycles.append({
        'battery_order': battery_order,
        'cycle_order': cycle_order,
        'c1a_I_dt': c1a_I_dt,
        'c1a_avg_T': c1a_avg_T,
        'c1a_avg_I': c1a_avg_I,
        'c1_max_I': c1_max_I,
        'c2_max_I': c2_max_I,
        'c1_max_T': c1_max_T,
        'c1_min_T': c1_min_T,
        'c2_max_T': c2_max_T,
        'c2_min_T': c2_min_T,
        'Qi': Qi,
    })


def print_cycles_csv(path):
    with open(path, 'w') as f:
        f.write(
            'battery_order,cycle_order,c1a_I_dt,c1a_avg_T,c1a_avg_I,c1_max_I,c2_max_I,c1_max_T,c1_min_T,c2_max_T,c2_min_T,Qi\n')
        for cycle in output_cycles:
            f.write(f"{cycle['battery_order']},{cycle['cycle_order']},{cycle['c1a_I_dt']},{cycle['c1a_avg_T']},{cycle['c1a_avg_I']},{cycle['c1_max_I']},{cycle['c2_max_I']},{cycle['c1_max_T']},{cycle['c1_min_T']},{cycle['c2_max_T']},{cycle['c2_min_T']},{cycle['Qi']}\n")

In [46]:
batch = f['batch_combined']

In [47]:
list(batch.keys())

['Vdlin',
 'barcode',
 'channel_id',
 'cycle_life',
 'cycles',
 'policy',
 'policy_readable',
 'summary']

In [48]:
def battery_info_iterator():
    battery_count = batch['policy'].shape[0]
    for battery_order in range(battery_count):
        policy = get_batch_cell_as_string(
            f[batch['policy'][battery_order, 0]][()])
        policy_readable = get_batch_cell_as_string(
            f[batch['policy_readable'][battery_order, 0]][()])
        # barcode = np.hstack(f[batch['barcode'][battery_order, 0]][()])
        barcode = ''
        cycle_life = np.hstack(f[batch['cycle_life'][battery_order, 0]][()])[0]
        cycle_count = f[batch['cycles'][battery_order, 0]]['I'].shape[0] - 1

        # barcode = np.hstack(
        #     list(map(lambda x:
        #              [int(x) >> 24, (x >> 16) & (int(2**8) - 1), (x >> 8)
        #               & (int(2**8) - 1), x & (int(2**8) - 1)],
        #              barcode)))

        # barcode = get_batch_cell_as_string(barcode)

        yield battery_order + 1, policy, policy_readable, barcode, cycle_life, cycle_count


# for battery_order, policy, policy_readable, barcode, cycle_life, cycle_count in battery_info_iterator():
#     print(
#         f'Battery {battery_order} - {policy_readable} - {barcode} - {cycle_life} - {cycle_count}')

In [49]:
for battery_order, policy, policy_readable, barcode, cycle_life, cycle_count in battery_info_iterator():
    pop_battery_info(battery_order, policy, policy_readable,
                     barcode, cycle_life, cycle_count)

In [50]:
def cycle_iterator(battery_order):
    # cl = f[batch['cycle_life'][0, 0]][()]
    cycles = f[batch['cycles'][battery_order - 1, 0]]

    for j in range(cycles['I'].shape[0]):
        I = np.hstack((f[cycles['I'][j, 0]][()]))
        Qc = np.hstack((f[cycles['Qc'][j, 0]][()]))
        Qd = np.hstack((f[cycles['Qd'][j, 0]][()]))
        Qdlin = np.hstack((f[cycles['Qdlin'][j, 0]][()]))
        T = np.hstack((f[cycles['T'][j, 0]][()]))
        Tdlin = np.hstack((f[cycles['Tdlin'][j, 0]][()]))
        V = np.hstack((f[cycles['V'][j, 0]][()]))

        dQdV = np.hstack((f[cycles['discharge_dQdV'][j, 0]][()]))
        t = np.hstack((f[cycles['t'][j, 0]][()]))

        yield I, Qc, Qd, Qdlin, T, Tdlin, V, dQdV, t


def cycle_find(battery_order, cycle_order):
    # cl = f[batch['cycle_life'][0, 0]][()]
    cycles = f[batch['cycles'][battery_order - 1, 0]]
    current_cycle_order = 0

    for j in range(cycles['I'].shape[0]):
        current_cycle_order = current_cycle_order + 1
        if current_cycle_order < cycle_order:
            continue

        I = np.hstack((f[cycles['I'][j, 0]][()]))
        Qc = np.hstack((f[cycles['Qc'][j, 0]][()]))
        Qd = np.hstack((f[cycles['Qd'][j, 0]][()]))
        Qdlin = np.hstack((f[cycles['Qdlin'][j, 0]][()]))
        T = np.hstack((f[cycles['T'][j, 0]][()]))
        Tdlin = np.hstack((f[cycles['Tdlin'][j, 0]][()]))
        V = np.hstack((f[cycles['V'][j, 0]][()]))

        dQdV = np.hstack((f[cycles['discharge_dQdV'][j, 0]][()]))
        t = np.hstack((f[cycles['t'][j, 0]][()]))

        return I, Qc, Qd, Qdlin, T, Tdlin, V, dQdV, t

In [51]:
battery_cycles_ouptput = []


def catch_invalid_cycle(battery_order, real_cycle, c1a_I_dt, c1a_avg_T, c1a_avg_I,
                        c1_max_I, c2_max_I, c1_max_T, c1_min_T, c2_max_T, c2_min_T, Qi, c1a_start_index, c1a_end_index,
                        end_c1_time, total_time):
    if c1a_I_dt <= 0.0001 or np.isnan(c1a_I_dt):
        print(f'Battery {battery_order} - Cycle {real_cycle} - c1a_I_dt <= 0')
        print(
            f'c1a_start_index: {c1a_start_index}, c1a_end_index: {c1a_end_index}, end_c1_time: {end_c1_time}, total_time: {total_time}')
        return True
    if c2_max_I <= 0.0001 or np.isnan(c2_max_I):
        print(f'Battery {battery_order} - Cycle {real_cycle} - c2_max_I <= 0')
        print(
            f'c1a_start_index: {c1a_start_index}, c1a_end_index: {c1a_end_index}, end_c1_time: {end_c1_time}, total_time: {total_time}')
        return True
    if c2_min_T <= 0.0001 or np.isnan(c2_min_T):
        print(f'Battery {battery_order} - Cycle {real_cycle} - c2_min_T <= 0')
        print(
            f'c1a_start_index: {c1a_start_index}, c1a_end_index: {c1a_end_index}, end_c1_time: {end_c1_time}, total_time: {total_time}')
        return True
    if c1_min_T <= 0.0001 or np.isnan(c1_min_T):
        print(f'Battery {battery_order} - Cycle {real_cycle} - c1_min_T <= 0')
        print(
            f'c1a_start_index: {c1a_start_index}, c1a_end_index: {c1a_end_index}, end_c1_time: {end_c1_time}, total_time: {total_time}')
        return True


battery = output_battery_infos[2]
print(battery)
battery_order = 9
real_cycle = 144

I, Qc, Qd, Qdlin, T, Tdlin, V, dQdV, t = cycle_find(
    battery_order=battery_order, cycle_order=real_cycle)

# print(f'Battery {battery_order} - Cycle {real_cycle}')

if (len(I) < 10):
    print('len(I) < 10', len(I))

Qc_max = np.max(Qc)
print(Qc_max)

if Qc_max < 0.88:
    print('Qc_max < 0.88', Qc_max)


current_cycle = real_cycle

cycle_order = current_cycle


Qd_max = np.max(Qd)

start_c1_time = 0
end_c1_time = 0

while end_c1_time < len(T) and Qc[end_c1_time] + 0.0001 > Qc_max:
    end_c1_time = end_c1_time + 1

print(str(end_c1_time) + "/" + str(len(T)))

while end_c1_time < len(T) and Qc[end_c1_time] + 0.00009 < Qc_max:
    end_c1_time = end_c1_time + 1

print(str(end_c1_time) + "/" + str(len(T)))

if end_c1_time >= len(T):
    print(f'Battery {battery_order} - Cycle {real_cycle}')
    print('end_c1_time >= len(T)', end_c1_time, "/", len(T))

c1_max_I = np.max(I[0:end_c1_time])
c2_max_I = np.max(abs(I[end_c1_time:]))

c1_max_T = np.max(T[0:end_c1_time])
c1_min_T = np.min(T[0:end_c1_time])
c2_max_T = np.max(T[end_c1_time:])
c2_min_T = np.min(T[end_c1_time:])


# TODO: Trong pha 1, khi gía trị hiệu điện thế từ 3.1V đến 3.3V, tính tổng I*Δt:


index = 0


while index < end_c1_time and V[index] <= 3.1:

    index = index + 1

c1a_start_index = index


while index < end_c1_time and V[index] < 3.3:

    index = index + 1

c1a_end_index = index


c1a_I_dt = np.sum(np.fromiter(
    (I[i] * (t[i] - t[i - 1]) for i in range(c1a_start_index, c1a_end_index)), float))
c1a_avg_T = np.mean(T[c1a_start_index:c1a_end_index])
c1a_avg_I = np.mean(I[c1a_start_index:c1a_end_index])

Qi = (Qd_max + Qc_max) / 2


# TODO: Xem xét liệu đến chu trình này thì Qi < 0.8*Qi lần sạc 1 đầu tiên? Nếu có thì pin hư.

catch_invalid_cycle(battery_order, real_cycle, c1a_I_dt, c1a_avg_T, c1a_avg_I,
                    c1_max_I, c2_max_I, c1_max_T, c1_min_T, c2_max_T, c2_min_T, Qi, c1a_start_index, c1a_end_index,
                    end_c1_time, len(T))

pop_cycle(battery_order, cycle_order, c1a_I_dt, c1a_avg_T, c1a_avg_I,
          c1_max_I, c2_max_I, c1_max_T, c1_min_T, c2_max_T, c2_min_T, Qi)

{'battery_order': 3, 'policy': '3_6C-80PER_3_6C', 'policy_readable': '3.6C(80%)-3.6C', 'barcode': '', 'cycle_life': 1177.0, 'cycle_count': 2235}
1.0860219
9/973
973/973
Battery 9 - Cycle 144
end_c1_time >= len(T) 973 / 973


ValueError: zero-size array to reduction operation maximum which has no identity